In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np
import math

In [ ]:
TRAIN_DATA_DIR = 'data/train/'
VALIDATION_DATA_DIR = 'data/val/'
TRAIN_SAMPLES = 500
VALIDATION_SAMPLES = 500
NUM_CLASSES = 2
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input,
                                  rotation_range = 20,
                                  width_shift_range = 0.2,
                                  height_shift_range = 0.2,
                                  zoom_range = 0.2)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train_generator = train_datagen.flow_from_directory(TRAIN_DATA_DIR,
                                                   target_size = (IMG_WIDTH, IMG_HEIGHT),
                                                   batch_size = BATCH_SIZE,
                                                   shuffle = True,
                                                   seed = 12345,
                                                   class_mode  = 'categorical')

validation_generator = val_datagen.flow_from_directory(VALIDATION_DATA_DIR,
                                                      target_size = (IMG_WIDTH, IMG_HEIGHT),
                                                      batch_size = BATCH_SIZE,
                                                      shuffle = False,
                                                      class_mode = 'categorical')

In [ ]:
def model_maker():
    base_model = MobileNet(include_top=False,
                           input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    
    for layer in base_model.layers[:]:
        # Freeze the layers of the base model
        layer.trainable = False 
    
    input = Input(shape = (IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation = 'relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation = 'softmax')(custom_model)
    
    return Model(inputs = input, outputs = predictions)

In [ ]:
model = model_maker()

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['acc'])

num_steps = math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE)

model.fit(train_generator,
                   steps_per_epoch = num_steps,
                   epochs = 10,
                   validation_data = validation_generator,
                   validation_steps = num_steps)

In [ ]:
model.save('model.h5')

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model = load_model('model.h5')

In [ ]:
img_path = '../sample-images/dog.jpg'
img = image.load_img(img_path, target_size = (224, 224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis = 0)
preprocessed_img = preprocess_input(expanded_img_array)
prediction = model.predict(preprocessed_img)
print(prediction)
print(validation_generator.class_indices)

In [ ]:
model = load_model('model.h5')

In [ ]:
img_path = '../sample-images/dog.jpg'
img = image.load_img(img_path, target_size = (224, 224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis = 0)
preprocessed_img = preprocess_input(expanded_img_array)
prediction = model.predict(preprocessed_img)
print(prediction)
print(validation_generator.class_indices)